# Testing out creating a Vector database with Chroma and Langchain

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma


In [ ]:
dict = {"example1": """Here is a synthetic summary of polling data in the UK:

Recent opinion polls in the United Kingdom have shown a tight race between the governing Conservative Party and the opposition Labour Party. The latest survey from YouGov has Labour holding a narrow 2-point lead over the Tories among decided voters, with 36% supporting Labour and 34% backing the Conservatives. However, other pollsters like Savanta ComRes have found the two parties running neck-and-neck at 33% each.

The smaller Liberal Democrat party is polling in the 12-15% range, while the Green Party and Reform UK are each capturing around 5-7% of the vote in most surveys. There are regional variations, with Labour performing more strongly in urban areas like London, while the Conservatives maintain advantages in many rural constituencies. 

On key issues, the public appears closely divided on the economy, with pluralities trusting Labour slightly more on cost of living but the Tories ahead on economic management. The Conservatives hold leads on immigration and crime, while Labour is seen as stronger on the NHS and education.

Looking ahead to the next general election, expected in 2024 or early 2025, polling averages currently put Labour and the Conservatives in a statistical tie nationally. However, due to the distribution of support, models suggest Labour may have a slight edge in being able to secure a governing majority. Of course, much could still change over the remaining years of this parliament.""",
"example2": """Here is another synthetic summary of polling data in the UK:

The latest opinion polling shows the Conservative Party, led by Prime Minister Rishi Sunak, holding a slender lead over the opposition Labour Party. According to an average compiled by Britain Elects, the Tories are at 34% among decided voters, with Labour trailing just behind at 32%.

However, there is significant variation between individual pollsters. A recent Redfield & Wilton survey put the Conservatives up 6 points, while a Survation poll had Labour narrowly ahead by 2%. The leftwing party appears to be benefiting from a squeeze on support for the Liberal Democrats, who are down to around 10%.

On the key issue of the economy, voters give the Conservatives only a slight edge in being trusted to handle it best. But Labour has opened up clear leads on priorities like the NHS, education, and the cost-of-living crisis according to many surveys.

Looking at leader favorability ratings, both Sunak and Labour's Keir Starmer remain unpopular overall, with roughly 60% viewing each unfavorably. However, Starmer scores better on attributes like competence and vision for the country.

If an election were held imminently, the polls suggest a hung parliament is one of the likelier outcomes, with no single party holding a majority. The Conservatives may hold a slight advantage in being able to form a governing coalition with smaller unionist parties.

Of course, these are just a snapshot in time and much could change in voting intentions over the remaining years before the next scheduled election in late 2024. The state of the economy will likely be the critical factor driving voters' choices."""}

for key, value in dict.items():
    with open(f"{key}.txt", "w") as file:
        file.write(value)


In [ ]:
# Load the document, split it into chunks, embed each chunk and load it into the vector store.
examples = ['example1.txt', 'example2.txt']
for doc in examples:
    raw_documents = TextLoader(doc).load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    documents = text_splitter.split_documents(raw_documents)
    print(documents)

In [ ]:
from langchain import hub
from langchain.llms import (
    HuggingFaceHub,
)
from langchain.embeddings import HuggingFaceEmbeddings 
import chromadb

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    ),
    # persist_directory="./chromadb",
)

In [ ]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 20})

In [ ]:
#fine tuned team used this prompt
prompt_template = """Given a query, return the a summary of the most similar descriptions on the polling descriptions contained in the contect provided.
 
### query: {query}

 
### Context: {context}

 
### Summary:
"""

In [ ]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    prompt_template
)
prompt

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnableParallel, RunnableLambda

chain = (
        {
        "context": itemgetter("query") 
            | retriever, 
            #| RunnableLambda(format_docs),
        "query": itemgetter("query"),
    }
    | prompt
   # | ft_llm
   # | StrOutputParser()
)
chain

In [ ]:
q = "What do young people think about policing"
print(chain.invoke({"query": q}))

In [ ]:
prompt = chain.invoke({"query": q}).to_string()

In [ ]:
print(prompt)